In [1]:
import requests
import humanize

In [2]:
base = "http://localhost:8771/api"

In [3]:
%%time
res = requests.get(base + "/lexical")
print(f"Response weighted {humanize.naturalsize(len(res.content))}")

Response weighted 12.7 MB
CPU times: user 23.6 ms, sys: 11.3 ms, total: 35 ms
Wall time: 3.39 s


In [4]:
def query(payload, **_payload):
    payload.update(_payload)
    res = requests.post(base + "/relations", json=payload)
    return res.json()

In [5]:
# Query for relations with VO source nodes
query({"source_type": "vo", "limit": 2})

[['vo:0000000', 'rdfs:subClassOf', 'vo:0000420'],
 ['vo:0000000', 'rdfs:subClassOf', 'vo:0000420']]

In [6]:
# Query for relations with SYMP target nodes
query({"target_type": "symp", "limit": 2})

[['symp:0000375', 'rdfs:subClassOf', 'symp:0000001'],
 ['doid:96', 'ro:0002452', 'symp:0000001']]

In [7]:
# Query for relations from DOID to SYMP nodes
query({"source_type": "doid", "target_type": "symp", "limit": 2})

[['doid:96', 'ro:0002452', 'symp:0000001'],
 ['doid:946', 'ro:0002452', 'symp:0000001']]

In [8]:
# Query for relations with a specific source
query({"source_curie": "doid:946", "limit": 2})

[['doid:946', 'ro:0002452', 'symp:0019177'],
 ['doid:946', 'ro:0002452', 'symp:0000570']]

In [9]:
# Query for relations with a specific target
query({"target_curie": "symp:0000570", "limit": 2})

[['symp:0020011', 'rdfs:subClassOf', 'symp:0000570'],
 ['symp:0000738', 'rdfs:subClassOf', 'symp:0000570']]

In [10]:
# Query for specific source + relation
query({"source_curie": "bfo:0000002", "relation": "rdfs:subClassOf", "limit": 2})

[['bfo:0000002', 'rdfs:subClassOf', 'bfo:0000001'],
 ['bfo:0000002', 'rdfs:subClassOf', 'bfo:0000001']]

In [11]:
# Query for specific source + relation over a finite number of hops
query(
    {
        "source_curie": "bfo:0000002",
        "relation": "rdfs:subClassOf",
        "relation_max_hops": 2,
        "limit": 2,
    }
)

[['bfo:0000002', ['rdfs:subClassOf'], 'bfo:0000001'],
 ['bfo:0000002', ['rdfs:subClassOf', 'rdfs:subClassOf'], 'owl:Thing']]

In [12]:
# Query for specific source + relation over a variable number of hops
query(
    {
        "source_curie": "bfo:0000002",
        "relation": "rdfs:subClassOf",
        "relation_max_hops": 0,
        "distinct": True,
    }
)

[['bfo:0000002', ['rdfs:subClassOf'], 'bfo:0000001'],
 ['bfo:0000002', ['rdfs:subClassOf', 'rdfs:subClassOf'], 'owl:Thing'],
 ['bfo:0000002', ['rdfs:subClassOf', 'bfo:0000108'], 'bfo:0000008'],
 ['bfo:0000002',
  ['rdfs:subClassOf', 'bfo:0000108', 'rdfs:subClassOf'],
  'bfo:0000003'],
 ['bfo:0000002',
  ['rdfs:subClassOf', 'bfo:0000108', 'rdfs:subClassOf', 'rdfs:subClassOf'],
  'bfo:0000001'],
 ['bfo:0000002',
  ['rdfs:subClassOf',
   'bfo:0000108',
   'rdfs:subClassOf',
   'rdfs:subClassOf',
   'rdfs:subClassOf'],
  'owl:Thing']]

In [13]:
# Get all ancestors of a given node by querying over paths of multiple queries
query(
    {
        "source_curie": "doid:946",
        "relation": ["rdfs:subClassOf", "part_of"],
        "relation_max_hops": 0,
        "distinct": True,
    }
)

[['doid:946', ['rdfs:subClassOf'], 'doid:2789'],
 ['doid:946', ['rdfs:subClassOf', 'rdfs:subClassOf'], 'doid:1398'],
 ['doid:946',
  ['rdfs:subClassOf', 'rdfs:subClassOf', 'rdfs:subClassOf'],
  'doid:0050117'],
 ['doid:946',
  ['rdfs:subClassOf', 'rdfs:subClassOf', 'rdfs:subClassOf', 'rdfs:subClassOf'],
  'doid:4'],
 ['doid:946',
  ['rdfs:subClassOf',
   'rdfs:subClassOf',
   'rdfs:subClassOf',
   'rdfs:subClassOf',
   'rdfs:subClassOf'],
  'bfo:0000016'],
 ['doid:946',
  ['rdfs:subClassOf',
   'rdfs:subClassOf',
   'rdfs:subClassOf',
   'rdfs:subClassOf',
   'rdfs:subClassOf',
   'rdfs:subClassOf'],
  'bfo:0000017'],
 ['doid:946',
  ['rdfs:subClassOf',
   'rdfs:subClassOf',
   'rdfs:subClassOf',
   'rdfs:subClassOf',
   'rdfs:subClassOf',
   'rdfs:subClassOf',
   'rdfs:subClassOf'],
  'bfo:0000020'],
 ['doid:946',
  ['rdfs:subClassOf',
   'rdfs:subClassOf',
   'rdfs:subClassOf',
   'rdfs:subClassOf',
   'rdfs:subClassOf',
   'rdfs:subClassOf',
   'rdfs:subClassOf',
   'rdfs:subClassOf

In [14]:
# Take any query and add full=True to get all information
# about nodes and edges - warning, though, it's getting big fast
query({"target_curie": "symp:0000570", "limit": 2, "full": True})

[[{'id': 'symp:0020011',
   'name': 'bloody diarrhea',
   'obsolete': False,
   'type': 'class'},
  {'graph': 'http://purl.obolibrary.org/obo/doid/imports/symp_import.owl',
   'pred': 'rdfs:subClassOf',
   'source': 'doid'},
  {'description': 'Diarrhea is a feces and droppng symptom involving the abnormally frequent intestinal evacuations with more or less fluid stools.',
   'id': 'symp:0000570',
   'name': 'diarrhea',
   'obsolete': False,
   'synonyms': ['the runs',
    'diarrhoea',
    'loose bowels',
    'loose bowel',
    'bacterial gastroenteritis',
    'fecal incontinence',
    'incontinence of feces'],
   'type': 'class',
   'xrefs': ['icd9cm:787.91', 'umls:C0011991', 'umls.aui:A0048148']}],
 [{'id': 'symp:0000738',
   'name': 'yellowish green diarrhea',
   'obsolete': False,
   'type': 'class'},
  {'graph': 'http://purl.obolibrary.org/obo/doid/imports/symp_import.owl',
   'pred': 'rdfs:subClassOf',
   'source': 'doid'},
  {'description': 'Diarrhea is a feces and droppng sympto